In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [28]:
import tensorflow as tf 
import time 
import os
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",
)
print("Done")


Done


Imported the file and uploaded the data of shakespeare text 

In [29]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(f"Length of text: {len(text)} characters")
print(text[:250])

Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



Put the data into 'Text' and checked the first 250 words and printed the total number of words 

In [30]:
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")
print(vocab)

65 unique characters
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Sorted all the data and pulled out the unique characters 

In [31]:
example_texts = ["abcdefg", "xyz"]

chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
chars
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

The text is vectorized here i.e. The letters are converted into vectors/tensors.

Created StringLookup layer from tensorflow using tf.keras.layers.StringLookup adn inputed in ids_from_chars 

In [32]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

It converts tokens from character to numeric ids to feed into the model .


In [33]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

This layer recovers the characters from the vectors of IDs, and returns them as a 'tf.RaggedTensor' of characters.since invert is true it helps in retrieving characters m  

In [34]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

characters are joined back

In [35]:
tf.strings.reduce_join(chars, axis=-1).numpy()
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

You can tf.strings.reduce_join to join the characters back into strings.

**Prediction task**
Creating training examples and targets

In [36]:

all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
all_ids


<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

 
Each unicode is converted to a corresponding id using utf endcoding and converted to tensors(Tensors arae stored as all_ids)
all_ids in next line displays the tensor/value in console of the program 

In [37]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode("utf-8"))

F
i
r
s
t
 
C
i
t
i


ids_dataset creates a dataset consisting of all the 

In [38]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


Batch method lets you easily convert these individual characters to sequences of the desired size.

In [39]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [40]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target(list("Tensorflow"))
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Dataset created as (input,label)by adjusting a pointer one space to the right and training the model to predict the next possible character(tensor)

In [41]:
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

The data is already spilt using tf.data earlier the batch size is getting defined and teh data is shuffled.


**Model Building**

In [42]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [43]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        # TODO - Create an embedding layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # TODO - Create a GRU layer
        self.gru = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        # TODO - Finally connect it with a dense layer
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        # since we are training a text generation model,
        # we use the previous state, in training. If there is no state,
        # then we initialize the state
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

* tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;

* tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.);

* tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

The layers are defined in MyModel class under tf.keras.Model 
embedding and GRU layers used and connected with the dense layer 
self embedding done in the lower function 



In [44]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

Mymodel provided to a variable 

In [45]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size)",
    )
    
model.summary()

(64, 100, 66) # (batch_size, sequence_length, vocab_size)
Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  16896     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [46]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0], num_samples=1
)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([57, 35, 38, 54, 64, 23, 60, 12, 16, 27, 23, 12, 62, 19, 37, 54, 27,
       59, 39, 31, 54, 20,  6, 33, 55, 54, 55,  8, 34, 33, 44, 27, 41,  3,
       31,  0, 43,  8, 21, 39,  9, 55, 58, 61, 24, 14, 11, 27, 38,  0, 20,
        2, 60,  6, 50, 56, 42, 44, 55, 26,  5, 26, 65, 41, 30, 28, 13, 23,
       24, 16, 17, 45, 16, 62, 37, 46, 53, 50,  3,  0, 51, 65, 41, 49, 49,
       36, 32, 50, 28, 38,  7, 44, 13, 38, 52, 23, 26, 61, 28,  3])

In [47]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'loathsome world,\nThan these poor compounds that thou mayst not sell.\nI sell thee poison; thou hast s'

Next Char Predictions:
 b"rVYoyJu;CNJ;wFXoNtZRoG'Tpop-UTeNb!R[UNK]d-HZ.psvKA:NY[UNK]G u'kqcepM&MzbQO?JKCDfCwXgnk![UNK]lzbjjWSkOY,e?YmJMvO!"


**Training the model**

In [48]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.188236, shape=(), dtype=float32)


Created a loss variable and used it on the last layer (shape);during initializatioan of model training the logits are supposed to be of close magnitudes to make sure the model is not overconfident while creating the early responses and the weights and biases initially not satisfactory enough .this ensures proper changes being registed while reducing the lost function.

In [49]:
tf.exp(example_batch_mean_loss).numpy()

65.90644

Checked Exponential mean to be approximately equal to the vocabulary size. Higher loss represents that model is sure about its wrong answers and the training is badly initialised.

In [50]:
model.compile(optimizer="adam", loss=loss)

Model is compiled using Adam optimiser and the loss is registered all along wih default arguments 

**Configuration of Checkpoints and Execution of training**

In [51]:
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True
)

Checkpoionts saved in a directory and 

In [52]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 12s 42ms/step - loss: 2.7320
Epoch 2/10
172/172 [==============================] - 9s 39ms/step - loss: 1.9977
Epoch 3/10
172/172 [==============================] - 9s 38ms/step - loss: 1.7154
Epoch 4/10
172/172 [==============================] - 9s 39ms/step - loss: 1.5508
Epoch 5/10
172/172 [==============================] - 9s 38ms/step - loss: 1.4511
Epoch 6/10
172/172 [==============================] - 9s 39ms/step - loss: 1.3832
Epoch 7/10
172/172 [==============================] - 9s 39ms/step - loss: 1.3308
Epoch 8/10
172/172 [==============================] - 9s 38ms/step - loss: 1.2857
Epoch 9/10
172/172 [==============================] - 9s 39ms/step - loss: 1.2447
Epoch 10/10
172/172 [==============================] - 9s 38ms/step - loss: 1.2045


Total number of epochs 10 to makesure the amount of time is monitored and not too much 

**Generation of text**

--------------------------------

In [53]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states
    
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
start = time.time()
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO: Pryay a knavour weep.

Third Gent. OVARD:
But I did scarvel to hear these vile?
Come, then thou art thy boots: let me do.

LADY ANNE:
Then such gone ease mey in but as ever ever:
looks there with a was to do at the fillage.

KING EDWARD IV:
Heaven buy most cold was Marcius to her
Some other, then.

Citiser:
No more, be cousin;
Which of these cormest first.

AUTOLYCUS:
He capets our conscretes: but not
arrive the battle pardon. This last, you have putting him:
A stetion will restore by mind.
Come, I will not made at the found to sit our fel:
The shepherdess of the fair presence: imposited me,
In deseral through can great world could not you know
Over'd on earth, my lord; concrule our chequed
That this the rube's blessing creature of the proport?

DUKE OF YORK:
King Ricemp! a sacrining of Exeter,
That now in peace is fair dead stemb'd King Henry's maid,
And do not move her.

HASTINGS:
I'd the counce of this costainly service,
I dare a pupure to himself as ever
Os friendly give you